## Wolffian duct differentiation - mesenchyme

In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc
import scrublet as scr
import seaborn as sns
import scipy.stats
import anndata
import os

import scipy as scipy
import scipy as sp
import pickle as pkl
import matplotlib.pyplot as plt
import re
from collections import defaultdict
from statsmodels.nonparametric.smoothers_lowess import lowess
from numpy import asarray as ar
from collections import Counter
import networkx as nx
import igraph
import glob

In [ ]:
print(anndata.__version__)

In [ ]:
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42

In [ ]:
males_post10pcw = sc.read('/nfs/team292/vl6/FetalReproductiveTract/males_post10pcw.20240326.h5ad')
males_post10pcw.shape

In [ ]:
sc.set_figure_params(scanpy=True, dpi=80, dpi_save=150, frameon=True, vector_friendly=True, 
                         fontsize=14, figsize=(7, 7), color_map=None, format='pdf', facecolor=None, transparent=False)

In [ ]:
sc.pl.umap(males_post10pcw, color="celltype")

In [ ]:
sc.pl.umap(males_post10pcw, color="celltype", groups = ['CaputEpididymis_Epithelium', 'Corpus/CaudaEpididymis_Epithelium',
                                                        'Corpus/CaputEpididymis_Mesenchyme', 'CaudaEpididymis_Mesenchyme', 
                                                        'VasDeferens_Mesenchyme'
                                                                         ])

In [ ]:
males_post10pcw = males_post10pcw[[i in [
                                                        'Corpus/CaputEpididymis_Mesenchyme',  
                                                        'VasDeferens_Mesenchyme']
                                    for i in males_post10pcw.obs['celltype']]]

In [ ]:
sc.pl.umap(males_post10pcw, color="celltype")

In [ ]:
import anndata

In [ ]:
adata = anndata.AnnData(X = males_post10pcw.raw.X, obs = males_post10pcw.obs, var = males_post10pcw.raw.var)
adata

In [ ]:
adata = adata[[i == 'G1' for i in adata.obs['phase']]]

In [ ]:
adata.obs['stage_pcw'].value_counts(dropna = False)

In [ ]:
# Random downsampling per cell type 
import random
from itertools import chain
def downsample(adata, labels, n): 
    
    myindex = adata.obs[labels].value_counts().index 
    myvalues = adata.obs[labels].value_counts().values
    clusters = pd.Series(myvalues, index = myindex)
    
    # Find clusters with > n cells 
    cl2downsample = clusters.index[ clusters.values > n ]

    # save all barcode ids from small clusters
    holder = []
    holder.append( adata.obs_names[[ i not in cl2downsample for i in adata.obs[labels] ]] ) 

    # randomly sample n cells in the cl2downsample
    for cl in cl2downsample:
        print(cl)
        cl_sample = adata[[ i == cl for i in adata.obs[labels]]].obs_names
        # n = int(round(len(cl_sample)/2, 0))
       
        cl_downsample = random.sample(set(cl_sample), n )
        holder.append(cl_downsample)
    
    # samples to include
    samples = list(chain(*holder))

    # Filter adata_count
    adata = adata[[ i in samples for i in adata.obs_names ]]
    return adata

In [ ]:
## import reproductive tract utils functions
import sys
cwd = '/nfs/team292/vl6/Experiments/Utils/'
sys.path.append(cwd)

import reptract_utils
import reptract_genes

In [ ]:
adata.shape

In [ ]:
adata.raw = adata.copy()

In [ ]:
sc.pp.filter_genes(adata, min_counts=10)

In [ ]:
adata = reptract_utils.normalize_log_transform(adata)

In [ ]:
adata = reptract_utils.hvgs_pca_umap(adata)

In [ ]:
sc.pl.umap(adata, color=['stage_pcw', 'sample', 
                         'n_genes', 'donor',  'doublet_scores',
                        'celltype', 'phase'], ncols = 2, wspace = 0.3, 
          color_map = 'OrRd')

In [ ]:
sc.pl.umap(adata, color = [ 'HOXA5', 'HOXA7', 'HOXA10', 'PTGER3', 'ACTA2', 'DES', 'MYH11',
                          'HOXA13', 'ISL1', 'TMEM163',  'WNT4', 'RXFP2', 'GDF10', 'COMP', 'CALY', 
                          'WNT2', 'SFRP2', 'CDH7', 'RAI2',  'HMGA2', 
                          'KRT18', 'HOXA11', 'HOXC10', 'SRD5A2', 'EMX2', 'FOXF1', 'FOXF2', 
                          'DLK1'], wspace = 0.15, use_raw = False,
           color_map = 'OrRd', ncols = 2)

In [ ]:
sc.tl.leiden(adata, resolution = 5)
sc.pl.umap(adata, color = 'leiden', legend_loc = 'on data')

In [ ]:
sc.pl.umap(adata, color = 'leiden', groups = ['62', '30', '41', '66'])

In [ ]:
adata = adata[[i not in ['62', '30', '41', '66'] for i in adata.obs['leiden']]]

In [ ]:
sc.pl.umap(adata, color = [ 'HOXA5', 'HOXA7', 'HOXA10', 'PTGER3', 'ACTA2', 'DES', 'MYH11',
                          'HOXA13', 'ISL1', 'TMEM163',  'WNT4', 'RXFP2', 'GDF10', 'COMP', 'CALY', 
                          'WNT2', 'SFRP2', 'CDH7', 'RAI2',  'HMGA2', 
                          'KRT18', 'HOXA11', 'HOXC10', 'SRD5A2', 'EMX2', 'FOXF1', 'FOXF2', 
                          'DLK1'], wspace = 0.15, use_raw = False,
           color_map = 'OrRd', ncols = 2)

In [ ]:
adata.obs['stage_pcw'].value_counts()

In [ ]:
# adata = downsample(adata, 'stage_pcw', 3000)

In [ ]:
adata = anndata.AnnData(X = adata.raw.X, var = adata.raw.var, obs = adata.obs)


In [ ]:
adata.raw = adata.copy()
sc.pp.filter_genes(adata, min_counts=10)
#adata = reptract_utils.per_gene_analysis(adata)
adata = reptract_utils.normalize_log_transform(adata)
adata = reptract_utils.hvgs_pca_umap(adata)

In [ ]:
sc.pl.umap(adata, color=['stage_pcw', 'sample', 
                         'n_genes', 'donor',  'doublet_scores',
                        'celltype'], ncols = 2, wspace = 0.3, 
          color_map = 'OrRd')

In [ ]:
sc.pl.umap(adata, color = ['HOXA5', 'HOXA7', 'HOXA10', 'PTGER3', 'ACTA2', 'DES', 'MYH11',
                          'HOXA13', 'ISL1', 'TMEM163',  'WNT4', 'RXFP2', 'GDF10', 'COMP', 'CALY', 
                          'WNT2', 'SFRP2', 'CDH7', 'RAI2',  'HMGA2', 
                          'KRT18', 'HOXA11', 'HOXC10', 'SRD5A2', 'EMX2', 'FOXF1', 'FOXF2', 
                          'DLK1'], wspace = 0.15, use_raw = False,
           color_map = 'OrRd', ncols = 2)

In [ ]:
import rpy2.rinterface_lib.callbacks
import logging
# Ignore R warning messages
#Note: this can be commented out to get more verbose R output
rpy2.rinterface_lib.callbacks.logger.setLevel(logging.ERROR)
import anndata2ri
anndata2ri.activate()
%load_ext rpy2.ipython

In [ ]:
# Extract PCA matrix and batch array
n_pcs = 18 # Principal components used to perform dimensionality reduction 
pca = adata.obsm['X_pca'][:, 0:(n_pcs)]
batch = adata.obs['donor'] # Batch ID, in this case stage

In [ ]:
%%R -i pca -i batch -o hem

library(harmony)
library(magrittr)
set.seed(1000)
hem <- HarmonyMatrix(pca, batch, theta=0, lambda = 6, do_pca=FALSE, verbose = FALSE)  
hem = data.frame(hem)

In [ ]:
# Add harmony values to the anndata object --> overwriting PCA 
adata.obsm['X_pca'] = hem.values 

In [ ]:
sc.pp.neighbors(adata, n_pcs = n_pcs, random_state= 123)
sc.tl.umap(adata, min_dist=0.4, spread = 1.2)

In [ ]:
sc.pl.umap(adata, color = ['HOXA5', 'HOXA7', 'HOXA9', 'HOXA10', 'PTGER3', 'ACTA2', 'DES', 'MYH11',
                          'HOXA13', 'ISL1', 'TMEM163',  'WNT4', 'RXFP2', 'GDF10', 'COMP', 'CALY', 
                          'WNT2', 'SFRP2', 'CDH7', 'RAI2',  'HMGA2', 
                          'KRT18', 'HOXA11', 'HOXC10', 'SRD5A2', 'EMX2', 'FOXF1', 'FOXF2', 
                          'DLK1'], wspace = 0.15, use_raw = False,
           color_map = 'OrRd', ncols = 2)

In [ ]:
sc.pl.umap(adata, color=['stage_pcw', 
                         'n_genes', 'donor',  'doublet_scores',
                        'celltype', 'phase'], ncols = 2, wspace = 0.3, 
          color_map = 'OrRd')

In [ ]:
sc.tl.leiden(adata, resolution = 0.5)
sc.pl.umap(adata, color = 'leiden', legend_loc = 'on data')

In [ ]:
sc.tl.draw_graph(adata)

In [ ]:
sc.pl.draw_graph(adata, color='leiden', legend_loc='on data')


In [ ]:
sc.pl.draw_graph(adata, color='n_genes', color_map='OrRd')


In [ ]:
sc.tl.paga(adata, groups='leiden')


In [ ]:
sc.pl.paga(adata, color=['leiden', 'HOXA9', 'HOXA7', 'HOXA10', 'HOXA11', 'RAI2'])


In [ ]:
sc.tl.draw_graph(adata, init_pos='paga')


In [ ]:
sc.pl.draw_graph(adata, color='leiden', legend_loc='on data')


In [ ]:
adata

In [ ]:
import anndata
bdata = anndata.AnnData(X = adata.raw.X, var = adata.raw.var, obs = adata.obs)

In [ ]:
import rpy2.rinterface_lib.callbacks
import logging
# Ignore R warning messages
#Note: this can be commented out to get more verbose R output
rpy2.rinterface_lib.callbacks.logger.setLevel(logging.ERROR)
import anndata2ri
anndata2ri.activate()
%load_ext rpy2.ipython

In [ ]:
for col in bdata.obs.columns: 
    if col != 'leiden':
        del bdata.obs[col]

In [ ]:
%%R -i bdata
bdata

In [ ]:
%%R -o mrks

library(SoupX)
counts <- assay(bdata, "X")
colnames(counts) <- colnames(bdata)
rownames(counts) <- rownames(bdata)
mrks = quickMarkers(counts, colData(bdata)$leiden, N = 50) # where clusters is the cell type assignment and 100 means I want the top 100 genes per cluster that pass the hypergeometric test

In [ ]:
sc.pl.umap(adata, color = ['leiden', 'stage_pcw'], legend_loc = 'on data')

In [ ]:
cluster = mrks[mrks['cluster'] == '0']['gene'].to_list()[0:9]
sc.pl.umap(adata, color = cluster, color_map = 'OrRd', ncols = 3, use_raw = False)

In [ ]:
cluster = mrks[mrks['cluster'] == '1']['gene'].to_list()[0:9]
sc.pl.umap(adata, color = cluster, color_map = 'OrRd', ncols = 3, use_raw = False)

In [ ]:
cluster = mrks[mrks['cluster'] == '2']['gene'].to_list()[0:9]
sc.pl.umap(adata, color = cluster, color_map = 'OrRd', ncols = 3, use_raw = False)

In [ ]:
cluster = mrks[mrks['cluster'] == '3']['gene'].to_list()[0:9]
sc.pl.umap(adata, color = cluster, color_map = 'OrRd', ncols = 3, use_raw = False)

In [ ]:
cluster = mrks[mrks['cluster'] == '4']['gene'].to_list()[0:9]
sc.pl.umap(adata, color = cluster, color_map = 'OrRd', ncols = 3, use_raw = False)

In [ ]:
cluster = mrks[mrks['cluster'] == '5']['gene'].to_list()[0:9]
sc.pl.umap(adata, color = cluster, color_map = 'OrRd', ncols = 3, use_raw = False)

In [ ]:
cluster = mrks[mrks['cluster'] == '6']['gene'].to_list()[0:9]
sc.pl.umap(adata, color = cluster, color_map = 'OrRd', ncols = 3, use_raw = False)

In [ ]:
cluster = mrks[mrks['cluster'] == '7']['gene'].to_list()[0:9]
sc.pl.umap(adata, color = cluster, color_map = 'OrRd', ncols = 3, use_raw = False)

In [ ]:
cluster = mrks[mrks['cluster'] == '8']['gene'].to_list()[0:9]
sc.pl.umap(adata, color = cluster, color_map = 'OrRd', ncols = 3, use_raw = False)

In [ ]:
sc.pl.umap(adata, color = 'leiden', legend_loc = 'on data')

In [ ]:
annots = {'0' : 'Epididymis_Mesenchyme', '1' : 'Epididymis_Mesenchyme', '2' : 'Epididymis_Mesenchyme',
         '3' : 'Epididymis_Mesenchyme', '4' : 'Epididymis_Mesenchyme','5' : 'Epididymis_Mesenchyme',
         '6' : 'Epididymis_Mesenchyme', '7' : 'VasDeferens_Mesenchyme', '8' : 'Epididymis_Mesenchyme'}

In [ ]:
adata.obs['mese_wolffian_lowres'] = adata.obs.leiden.map(annots)

In [ ]:
sc.pl.umap(adata, color = ['stage_pcw', 'donor', 'leiden', 'mese_wolffian_lowres'], 
          ncols = 2, save = '_mese_wolffian')

In [ ]:
#del adata.obs['epi_mullerian_lowres']
#del adata.obs['epi_mullerian_highres']

In [ ]:
adata.write('/nfs/team292/vl6/FetalReproductiveTract/wolffian_mese_late_post10pcw.h5ad')

In [ ]:
adata = sc.read('/nfs/team292/vl6/FetalReproductiveTract/mullerian_mese_late_post10pcw.h5ad')

In [ ]:
adata.shape